In [3]:
from __future__ import division, print_function
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
from keras.layers import Input, Conv1D, BatchNormalization, Activation, Dropout, MaxPooling1D, Lambda, add, Dense, TimeDistributed
from keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout, Dense, add, GlobalAveragePooling1D, Reshape
from keras.models import Model
from keras.optimizers import Adam
from config import get_config


In [8]:
def ECG_model(config):
    inputs = Input(shape=(config.input_size, 1), name='input')
    
    x = Conv1D(filters=config.filter_length, kernel_size=config.kernel_size, padding='same', strides=1, kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    for i in range(15):  # Your loop for convolutional and identity blocks
        shortcut = x
        if i % 2 == 0:
            x = Conv1D(filters=config.filter_length, kernel_size=1, strides=2, padding='same', kernel_initializer='he_normal')(x)
        else:
            x = Conv1D(filters=config.filter_length, kernel_size=config.kernel_size, padding='same', strides=1, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(config.drop_rate)(x)
        x = Conv1D(filters=config.filter_length, kernel_size=config.kernel_size, padding='same', strides=1, kernel_initializer='he_normal')(x)
        
        if i % 2 == 0:
            shortcut = Conv1D(filters=config.filter_length, kernel_size=1, strides=2, padding='same', kernel_initializer='he_normal')(shortcut)
        x = add([x, shortcut])
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    
    # Adjusting for TimeDistributed layer
    # Assuming we need to adjust the dimensionality for TimeDistributed
    x = Reshape((-1, config.filter_length))(x)  # Adjust the reshape dimensions as necessary

    classes = ['N', 'V', '/', 'A', 'F', '~']
    len_classes = len(classes)
    outputs = TimeDistributed(Dense(len_classes, activation='softmax'))(x)

    model = Model(inputs=inputs, outputs=outputs)
    adam = Adam(lr=config.min_lr)  # Adjusted to use min_lr from your config
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model